In [1]:
# !pip install transformers==4.44.2

In [2]:
!git clone https://github.com/qhungngo/EVBCorpus.git

fatal: destination path 'EVBCorpus' already exists and is not an empty directory.


In [3]:
!pip install rarfile

In [4]:
import re
import rarfile
import os
import unicodedata
from transformers import MarianTokenizer, MarianMTModel, Trainer, TrainingArguments
from datasets import Dataset

2025-10-21 10:01:14.551491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761040874.573717    2203 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761040874.580524    2203 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
rar_path = "/kaggle/working/EVBCorpus/EVBCorpus_EVBNews_v2.0.rar"
extract_dir = "/kaggle/working/data"

# Tạo thư mục đích nếu chưa có
os.makedirs(extract_dir, exist_ok=True)

# Mở và giải nén
with rarfile.RarFile(rar_path) as rf:
    rf.extractall(path=extract_dir)

print("✅ Giải nén thành công!")

✅ Giải nén thành công!


In [6]:
# import shutil
# shutil.rmtree('/kaggle/working/data/EVBCorpus_v1')


In [7]:
import os
from bs4 import BeautifulSoup

# Đường dẫn tới thư mục chứa các file SGML
data_dir = "/kaggle/working/data"

# Lưu các bài báo
articles = []

# Duyệt qua tất cả file .sgml
for filename in sorted(os.listdir(data_dir)):
    if filename.endswith(".sgml"):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()
            soup = BeautifulSoup(content, "html.parser")

            # Tùy cấu trúc file, có thể là <DOC> hoặc <TEXT>
            for doc in soup.find_all("doc"):
                docno = doc.find("docno").get_text(strip=True) if doc.find("docno") else None
                text = doc.find("text").get_text(strip=True) if doc.find("text") else None

                if text:
                    articles.append({"id": docno or filename, "text": text})

print(f"Đã đọc {len(articles)} bài báo.")
print(articles[0])  # xem thử 1 mẫu


Đã đọc 1000 bài báo.
{'id': 'N0001.sgml', 'text': 'What is a Fenqing ?Fenqing là gì ?1-3;2-2;4-1;Fenqing is a Chinese word which literally means " angry youth " .Fenqing là một từ tiếng Hoa mà nghĩa đen là " thanh niên phẫn nộ " .1-1;2-2;3-3;4-5,6;5-4;6-7;7-8,9;8-10;10-14,15;11-12,13;This word has many translations in English such as cynical youth , young nationalists , hysterical youth and angry young men .Từ này có nhiều cách dịch sang tiếng Anh như là thanh niên hoài nghi , thanh niên theo chủ nghĩa dân tộc , thanh niên cuồng loạn và thanh niên tức giận .1-2;2-1;3-3;4-4;5-5,6;6-7;7-8,9;8-10,11;10-14,15;11-12,13;13-17,18;14-20,21,22,23;16-27,28;17-25,26;18-29;19-32,33;20-30,31;I personally like to call them mob youth or ignorant angry youth .Cá nhân tôi thích gọi chúng là bọn thanh niên du thủ du thực hoặc thanh niên phẫn nộ và ngu dốt .1-3;2-1,2;3-4;5-5;6-6;7-11,12,13,14;8-9,10;9-15;10-21,22;11-18,19;12-16,17;It is impossible to understand China without knowing what Fenqing is and w

In [8]:
def extract_sentence_pairs(text):
    # Tách theo dấu chấm câu và căn chỉnh
    text = re.sub(r'([\d,]+)-([\d,]+);', '', text)
    parts = re.split(r'(?<=[\.\?\!])', text)
    parts = [p.strip() for p in parts if p.strip()]

    src, tgt = [], []
    for i in range(0, len(parts), 2):
        if i + 1 < len(parts):
            en = parts[i]
            vi = parts[i + 1]
            # Bỏ qua dòng căn chỉnh (phần thứ 2 sau tiếng Việt)
            if re.search(r'[A-Za-z]', en) and re.search(r'[À-ỹ]', vi):
                src.append(en)
                tgt.append(vi)
    return src, tgt


In [9]:
def normalize_text(s):
    s = unicodedata.normalize('NFC', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

In [10]:
sources, targets = [], []
for article in articles:
    src, tgt = extract_sentence_pairs(article['text'])
    sources.extend([normalize_text(s) for s in src])
    targets.extend([normalize_text(t) for t in tgt])

In [11]:
data = {"translation": [{"en": s, "vi": t} for s, t in zip(sources, targets)]}
dataset = Dataset.from_dict(data)

In [12]:
# Load mô hình pretrained
model_name = "Helsinki-NLP/opus-mt-en-vi"
tokenizer = MarianTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = MarianMTModel.from_pretrained(model_name, trust_remote_code=True)

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [14]:
def preprocess_function(batch):
    inputs = [ex["en"] for ex in batch["translation"]]
    targets = [ex["vi"] for ex in batch["translation"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/27581 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
args = TrainingArguments(
    output_dir="./results",
    logging_strategy="steps",
    logging_steps=50,             # tần suất log
    save_strategy="steps",        # lưu checkpoint định kỳ
    save_steps=500,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    save_total_limit=1,
    logging_dir="./logs",
    report_to="none"
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
50,0.647500
100,0.629400
150,0.610700
200,0.579700
250,0.615200
300,0.628600
350,0.656500
400,0.615900
450,0.594800
500,0.571000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[53684]], 'forced_eos_token_id': 0}
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, '

KeyboardInterrupt: 

In [28]:
text = ["With all sincerity, I wish you will be happy."]
inputs = tokenizer(text, return_tensors="pt", padding=True).to("cuda")
translated = model.generate(**inputs)
print(tokenizer.decode(translated[0], skip_special_tokens=True))

Với tất cả sự thành thật, tôi ước gì bạn sẽ hạnh phúc.
